# Demo for using C3S intake catalog

Intake Example:
https://github.com/intake/intake-examples/blob/master/tutorial/data_scientist.ipynb


In [1]:
import intake

## Open remote catalog

In [2]:
cat_url = "https://raw.githubusercontent.com/cp4cds/c3s_34g_manifests/master/intake/catalogs/c3s.yaml"

cat = intake.open_catalog(cat_url)
list(cat)

['c3s-cmip5',
 'c3s-cmip5-daily-pressure-level',
 'c3s-cmip5-daily-single-level',
 'c3s-cmip5-monthly-pressure-level',
 'c3s-cmip5-monthly-single-level',
 'c3s-cmip6',
 'c3s-cmip6-decadal',
 'c3s-cmip6-decadal-dwd',
 'c3s-cordex']

## Load catalog for c3s-cmip6
Catalogs will be cached locally in `~/.intake/cache`.

See: https://intake.readthedocs.io/en/latest/catalog.html?highlight=simplecache#caching-source-files-locally

In [3]:
df_cmip6 = cat['c3s-cmip6'].read()
df_cmip6

,ds_id,path,size,mip_era,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,start_time,end_time,bbox,level
0,c3s-cmip6.ScenarioMIP.CAS.FGOALS-g3.ssp119.r1i...,ScenarioMIP/CAS/FGOALS-g3/ssp119/r1i1p1f1/day/...,21053432,c3s-cmip6,ScenarioMIP,CAS,FGOALS-g3,ssp119,r1i1p1f1,day,sfcWind,gn,v20191202,2086-01-01T12:00:00,2086-12-31T12:00:00,"0.00, -90.00, 358.00, 90.00",10.00
1,c3s-cmip6.ScenarioMIP.EC-Earth-Consortium.EC-E...,ScenarioMIP/EC-Earth-Consortium/EC-Earth3-Veg-...,1691093,c3s-cmip6,ScenarioMIP,EC-Earth-Consortium,EC-Earth3-Veg-LR,ssp585,r1i1p1f1,SImon,sithick,gn,v20201201,2091-01-16T12:00:00,2091-12-16T12:00:00,"0.05, -78.58, 359.99, 89.74",NaN
2,c3s-cmip6.CMIP.EC-Earth-Consortium.EC-Earth3-V...,CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/hist...,65838336,c3s-cmip6,CMIP,EC-Earth-Consortium,EC-Earth3-Veg-LR,historical,r1i1p1f1,day,huss,gr,v20200217,1855-01-01T12:00:00,1855-12-31T12:00:00,"0.00, -89.14, 358.88, 89.14",2.00
3,c3s-cmip6.ScenarioMIP.EC-Earth-Consortium.EC-E...,ScenarioMIP/EC-Earth-Consortium/EC-Earth3-CC/s...,167116577,c3s-cmip6,ScenarioMIP,EC-Earth-Consortium,EC-Earth3-CC,ssp245,r1i1p1f1,day,huss,gr,v20210113,2066-01-01T12:00:00,2066-12-31T12:00:00,"0.00, -89.46, 359.30, 89.46",2.00
4,c3s-cmip6.CMIP.EC-Earth-Consortium.EC-Earth3-C...,CMIP/EC-Earth-Consortium/EC-Earth3-CC/historic...,142104415,c3s-cmip6,CMIP,EC-Earth-Consortium,EC-Earth3-CC,historical,r1i1p1f1,day,pr,gr,v20210113,1961-01-01T12:00:00,1961-12-31T12:00:00,"0.00, -89.46, 359.30, 89.46",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133212,c3s-cmip6.CMIP.EC-Earth-Consortium.EC-Earth3-A...,CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/his...,5737958,c3s-cmip6,CMIP,EC-Earth-Consortium,EC-Earth3-AerChem,historical,r1i1p1f1,Amon,hfss,gr,v20200624,1913-01-16T12:00:00,1913-12-16T12:00:00,"0.00, -89.46, 359.30, 89.46",NaN
133213,c3s-cmip6.CMIP.AWI.AWI-ESM-1-1-LR.historical.r...,CMIP/AWI/AWI-ESM-1-1-LR/historical/r1i1p1f1/LI...,78044,c3s-cmip6,CMIP,AWI,AWI-ESM-1-1-LR,historical,r1i1p1f1,LImon,snw,gn,v20200212,1967-01-16T12:00:00,1967-12-16T12:00:00,"0.00, -88.57, 358.12, 88.57",NaN
133214,c3s-cmip6.ScenarioMIP.CAS.FGOALS-g3.ssp126.r1i...,ScenarioMIP/CAS/FGOALS-g3/ssp126/r1i1p1f1/day/...,21053188,c3s-cmip6,ScenarioMIP,CAS,FGOALS-g3,ssp126,r1i1p1f1,day,psl,gn,v20200928,2051-01-01T12:00:00,2051-12-31T12:00:00,"0.00, -90.00, 358.00, 90.00",NaN
133215,c3s-cmip6.ScenarioMIP.EC-Earth-Consortium.EC-E...,ScenarioMIP/EC-Earth-Consortium/EC-Earth3/ssp1...,4745936,c3s-cmip6,ScenarioMIP,EC-Earth-Consortium,EC-Earth3,ssp119,r4i1p1f1,Amon,ts,gr,v20200425,2039-01-16T12:00:00,2039-12-16T12:00:00,"0.00, -89.46, 359.30, 89.46",NaN


### Search dataset ...

In [4]:
df = df_cmip6.loc[
    (df_cmip6.variable_id=="tas") 
    & (df_cmip6.experiment_id=="historical")
    & (df_cmip6.table_id=="day")
    & (df_cmip6.member_id=="r1i1p1f1")
    & (df_cmip6.institution_id=="MIROC")
]
df.head()

,ds_id,path,size,mip_era,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,start_time,end_time,bbox,level
48328,c3s-cmip6.CMIP.MIROC.MIROC6.historical.r1i1p1f...,CMIP/MIROC/MIROC6/historical/r1i1p1f1/day/tas/...,268131835,c3s-cmip6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,day,tas,gn,v20191016,1850-01-01T12:00:00,1859-12-31T12:00:00,"0.00, -88.93, 358.59, 88.93",2.00
49075,c3s-cmip6.CMIP.MIROC.MIROC6.historical.r1i1p1f...,CMIP/MIROC/MIROC6/historical/r1i1p1f1/day/tas/...,268260608,c3s-cmip6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,day,tas,gn,v20191016,1860-01-01T12:00:00,1869-12-31T12:00:00,"0.00, -88.93, 358.59, 88.93",2.00
50196,c3s-cmip6.CMIP.MIROC.MIROC6.historical.r1i1p1f...,CMIP/MIROC/MIROC6/historical/r1i1p1f1/day/tas/...,268187514,c3s-cmip6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,day,tas,gn,v20191016,1870-01-01T12:00:00,1879-12-31T12:00:00,"0.00, -88.93, 358.59, 88.93",2.00
52562,c3s-cmip6.CMIP.MIROC.MIROC6.historical.r1i1p1f...,CMIP/MIROC/MIROC6/historical/r1i1p1f1/day/tas/...,268278571,c3s-cmip6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,day,tas,gn,v20191016,1880-01-01T12:00:00,1889-12-31T12:00:00,"0.00, -88.93, 358.59, 88.93",2.00
54490,c3s-cmip6.CMIP.MIROC.MIROC6.historical.r1i1p1f...,CMIP/MIROC/MIROC6/historical/r1i1p1f1/day/tas/...,268214955,c3s-cmip6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,day,tas,gn,v20191016,1890-01-01T12:00:00,1899-12-31T12:00:00,"0.00, -88.93, 358.59, 88.93",2.00


In [5]:
df.ds_id.unique()

array(['c3s-cmip6.CMIP.MIROC.MIROC6.historical.r1i1p1f1.day.tas.gn.v20191016'],
      dtype=object)

## Load catalog for c3s-cmip6-decadal

In [6]:
df_cmip6_decadal = cat['c3s-cmip6-decadal'].read()
df_cmip6_decadal

,ds_id,path,size,mip_era,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,start_time,end_time,bbox,level
0,c3s-cmip6.DCPP.MPI-M.MPI-ESM1-2-HR.dcppA-hindc...,DCPP/MPI-M/MPI-ESM1-2-HR/dcppA-hindcast/s2016-...,492479692,c3s-cmip6,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,s2016-r8i1p1f1,day,tasmax,gn,v20210111,2016-11-01T12:00:00,2026-12-31T12:00:00,"0.00, -89.28, 359.06, 89.28",2.00
1,c3s-cmip6.DCPP.MPI-M.MPI-ESM1-2-HR.dcppA-hindc...,DCPP/MPI-M/MPI-ESM1-2-HR/dcppA-hindcast/s2009-...,737204109,c3s-cmip6,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,s2009-r8i1p1f1,day,pr,gn,v20210107,2009-11-01T12:00:00,2019-12-31T12:00:00,"0.00, -89.28, 359.06, 89.28",NaN
2,c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hind...,DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1987...,953384,c3s-cmip6,DCPP,MOHC,HadGEM3-GC31-MM,dcppA-hindcast,s1987-r3i1p1f2,Amon,pr,gn,v20200417,1987-11-16T00:00:00,1987-12-16T00:00:00,"0.42, -89.72, 359.58, 89.72",NaN
3,c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hind...,DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1995...,603757,c3s-cmip6,DCPP,MOHC,HadGEM3-GC31-MM,dcppA-hindcast,s1995-r6i1p1f2,Amon,tas,gn,v20200417,1995-11-16T00:00:00,1995-12-16T00:00:00,"0.42, -89.72, 359.58, 89.72",1.50
4,c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hind...,DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s2004...,14824913,c3s-cmip6,DCPP,MOHC,HadGEM3-GC31-MM,dcppA-hindcast,s2004-r10i1p1f2,AERday,zg500,gn,v20200417,2004-11-01T12:00:00,2004-12-30T12:00:00,"0.42, -89.72, 359.58, 89.72",50000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102225,c3s-cmip6.DCPP.DWD.MPI-ESM1-2-LR.dcppA-hindcas...,DCPP/DWD/MPI-ESM1-2-LR/dcppA-hindcast/s1995-r1...,4119531,c3s-cmip6,DCPP,DWD,MPI-ESM1-2-LR,dcppA-hindcast,s1995-r16i1p1f1,Amon,tas,gn,v20220126,1995-11-30T18:00:00,2005-12-31T18:00:00,"0.00, -88.57, 358.12, 88.57",2.00
102226,c3s-cmip6.DCPP.DWD.MPI-ESM1-2-LR.dcppA-hindcas...,DCPP/DWD/MPI-ESM1-2-LR/dcppA-hindcast/s1973-r1...,6635233,c3s-cmip6,DCPP,DWD,MPI-ESM1-2-LR,dcppA-hindcast,s1973-r16i1p1f1,Amon,pr,gn,v20220126,1973-11-30T18:00:00,1983-12-31T18:00:00,"0.00, -88.57, 358.12, 88.57",NaN
102227,c3s-cmip6.DCPP.DWD.MPI-ESM1-2-LR.dcppA-hindcas...,DCPP/DWD/MPI-ESM1-2-LR/dcppA-hindcast/s2014-r8...,4114536,c3s-cmip6,DCPP,DWD,MPI-ESM1-2-LR,dcppA-hindcast,s2014-r8i1p1f1,Amon,tas,gn,v20220126,2014-11-30T18:00:00,2024-12-31T18:00:00,"0.00, -88.57, 358.12, 88.57",2.00
102228,c3s-cmip6.DCPP.DWD.MPI-ESM1-2-LR.dcppA-hindcas...,DCPP/DWD/MPI-ESM1-2-LR/dcppA-hindcast/s2006-r1...,6641928,c3s-cmip6,DCPP,DWD,MPI-ESM1-2-LR,dcppA-hindcast,s2006-r14i1p1f1,Amon,pr,gn,v20220126,2006-11-30T18:00:00,2016-12-31T18:00:00,"0.00, -88.57, 358.12, 88.57",NaN


In [14]:
df = df_cmip6_decadal.loc[
    (df_cmip6_decadal.variable_id=="tas") 
    & (df_cmip6_decadal.experiment_id=="dcppA-hindcast")
    & (df_cmip6_decadal.table_id=="Amon")
    & (df_cmip6_decadal.institution_id=="MOHC")
    & (df_cmip6_decadal.start_time=="1995-11-16T00:00:00")
]
df.nunique()

ds_id             10
path              10
size              10
mip_era            1
activity_id        1
institution_id     1
source_id          1
experiment_id      1
member_id         10
table_id           1
variable_id        1
grid_label         1
version            1
start_time         1
end_time           1
bbox               1
level              1
dtype: int64

In [24]:
ds_ids = list(df.ds_id)
ds_ids.sort()
ds_ids

['c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r1i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r2i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r3i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r4i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r5i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r6i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r7i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r8i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r9i1p1f2.Amon.tas.gn.v20200417']

## Load Catalog for C3S-CORDEX

In [8]:
df_cordex = cat['c3s-cordex'].read()
df_cordex

,ds_id,path,size,project,product,domain,institute,driving_model,experiment,ensemble,rcm_name,rcm_version,time_frequency,variable,version,start_time,end_time,bbox,level,experiment_id
0,c3s-cordex.output.AFR-22.ICTP.MOHC-HadGEM2-ES....,output/AFR-22/ICTP/MOHC-HadGEM2-ES/rcp85/r1i1p...,21597412,c3s-cordex,output,AFR-22,ICTP,MOHC-HadGEM2-ES,rcp85,r1i1p1,ICTP-RegCM4-7,v0,day,uas,v20201015,2005-12-01T12:00:00,2005-12-30T12:00:00,"-41.54, -49.92, 65.54, 47.28",10.00,NaN
1,c3s-cordex.output.AFR-22.ICTP.MPI-M-MPI-ESM-MR...,output/AFR-22/ICTP/MPI-M-MPI-ESM-MR/rcp26/r1i1...,19464039,c3s-cordex,output,AFR-22,ICTP,MPI-M-MPI-ESM-MR,rcp26,r1i1p1,ICTP-RegCM4-7,v0,day,hurs,v20201015,2005-12-01T12:00:00,2005-12-31T12:00:00,"-41.54, -49.92, 65.54, 47.28",2.00,NaN
2,c3s-cordex.output.AFR-22.ICTP.MOHC-HadGEM2-ES....,output/AFR-22/ICTP/MOHC-HadGEM2-ES/rcp26/r1i1p...,14791855,c3s-cordex,output,AFR-22,ICTP,MOHC-HadGEM2-ES,rcp26,r1i1p1,ICTP-RegCM4-7,v0,day,tas,v20201015,2005-12-01T12:00:00,2005-12-30T12:00:00,"-41.54, -49.92, 65.54, 47.28",2.00,NaN
3,c3s-cordex.output.AFR-22.ICTP.MPI-M-MPI-ESM-MR...,output/AFR-22/ICTP/MPI-M-MPI-ESM-MR/historical...,236295975,c3s-cordex,output,AFR-22,ICTP,MPI-M-MPI-ESM-MR,historical,r1i1p1,ICTP-RegCM4-7,v0,day,rsds,v20201015,1970-01-01T12:00:00,1970-12-31T12:00:00,"-41.54, -49.92, 65.54, 47.28",NaN,NaN
4,c3s-cordex.output.AFR-22.ICTP.MOHC-HadGEM2-ES....,output/AFR-22/ICTP/MOHC-HadGEM2-ES/rcp85/r1i1p...,18728887,c3s-cordex,output,AFR-22,ICTP,MOHC-HadGEM2-ES,rcp85,r1i1p1,ICTP-RegCM4-7,v0,day,hurs,v20201015,2005-12-01T12:00:00,2005-12-30T12:00:00,"-41.54, -49.92, 65.54, 47.28",2.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566767,c3s-cordex.output.EUR-11.CNRM.CNRM-CERFACS-CNR...,output/EUR-11/CNRM/CNRM-CERFACS-CNRM-CM5/rcp85...,1129745566,c3s-cordex,output,EUR-11,CNRM,CNRM-CERFACS-CNRM-CM5,NaN,r1i1p1,CNRM-ALADIN63,v2,day,sfcWind,v20190419,2091-01-01T12:00:00,2095-12-31T12:00:00,"-51.56, 20.98, 72.56, 73.97",NaN,rcp85
566768,c3s-cordex.output.EUR-11.ICTP.MPI-M-MPI-ESM-LR...,output/EUR-11/ICTP/MPI-M-MPI-ESM-LR/rcp26/r1i1...,1448576089,c3s-cordex,output,EUR-11,ICTP,MPI-M-MPI-ESM-LR,NaN,r1i1p1,ICTP-RegCM4-6,v1,3hr,rsus,v20190502,2012-01-01T01:30:00,2012-12-31T22:30:00,"-62.74, 14.77, 82.24, 76.33",NaN,rcp26
566769,c3s-cordex.output.EUR-11.ICTP.ICHEC-EC-EARTH.r...,output/EUR-11/ICTP/ICHEC-EC-EARTH/rcp85/r12i1p...,294010575,c3s-cordex,output,EUR-11,ICTP,ICHEC-EC-EARTH,NaN,r12i1p1,ICTP-RegCM4-6,v1,day,rsus,v20190502,2009-01-01T12:00:00,2009-12-31T12:00:00,"-62.74, 14.77, 82.24, 76.33",NaN,rcp85
566770,c3s-cordex.output.EUR-11.CNRM.CNRM-CERFACS-CNR...,output/EUR-11/CNRM/CNRM-CERFACS-CNRM-CM5/rcp85...,1129918622,c3s-cordex,output,EUR-11,CNRM,CNRM-CERFACS-CNRM-CM5,NaN,r1i1p1,CNRM-ALADIN63,v2,day,sfcWind,v20190419,2096-01-01T12:00:00,2100-12-31T12:00:00,"-51.56, 20.98, 72.56, 73.97",NaN,rcp85


In [9]:
df = df_cordex.loc[
    (df_cordex.variable=="tas") 
    & (df_cordex.experiment=="rcp85")
    & (df_cordex.domain=="EUR-11")
    & (df_cordex.time_frequency=="mon")
    & (df_cordex.driving_model=="MOHC-HadGEM2-ES")
    & (df_cordex.ensemble=="r1i1p1")
]
df.head()

,ds_id,path,size,project,product,domain,institute,driving_model,experiment,ensemble,rcm_name,rcm_version,time_frequency,variable,version,start_time,end_time,bbox,level,experiment_id
107046,c3s-cordex.output.EUR-11.CLMcom.MOHC-HadGEM2-E...,output/EUR-11/CLMcom/MOHC-HadGEM2-ES/rcp85/r1i...,25751621,c3s-cordex,output,EUR-11,CLMcom,MOHC-HadGEM2-ES,rcp85,r1i1p1,CLMcom-CCLM4-8-17,v1,mon,tas,v20150320,2006-01-16T00:00:00,2010-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.58",2.00,NaN
107097,c3s-cordex.output.EUR-11.CLMcom.MOHC-HadGEM2-E...,output/EUR-11/CLMcom/MOHC-HadGEM2-ES/rcp85/r1i...,47693820,c3s-cordex,output,EUR-11,CLMcom,MOHC-HadGEM2-ES,rcp85,r1i1p1,CLMcom-CCLM4-8-17,v1,mon,tas,v20150320,2011-01-16T00:00:00,2020-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.58",2.00,NaN
107138,c3s-cordex.output.EUR-11.CLMcom.MOHC-HadGEM2-E...,output/EUR-11/CLMcom/MOHC-HadGEM2-ES/rcp85/r1i...,47688544,c3s-cordex,output,EUR-11,CLMcom,MOHC-HadGEM2-ES,rcp85,r1i1p1,CLMcom-CCLM4-8-17,v1,mon,tas,v20150320,2021-01-16T00:00:00,2030-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.58",2.00,NaN
107177,c3s-cordex.output.EUR-11.CLMcom.MOHC-HadGEM2-E...,output/EUR-11/CLMcom/MOHC-HadGEM2-ES/rcp85/r1i...,47704709,c3s-cordex,output,EUR-11,CLMcom,MOHC-HadGEM2-ES,rcp85,r1i1p1,CLMcom-CCLM4-8-17,v1,mon,tas,v20150320,2031-01-16T00:00:00,2040-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.58",2.00,NaN
107220,c3s-cordex.output.EUR-11.CLMcom.MOHC-HadGEM2-E...,output/EUR-11/CLMcom/MOHC-HadGEM2-ES/rcp85/r1i...,47661324,c3s-cordex,output,EUR-11,CLMcom,MOHC-HadGEM2-ES,rcp85,r1i1p1,CLMcom-CCLM4-8-17,v1,mon,tas,v20150320,2041-01-16T00:00:00,2050-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.58",2.00,NaN


In [10]:
collection = df.ds_id.unique()[0]
collection

'c3s-cordex.output.EUR-11.CLMcom.MOHC-HadGEM2-ES.rcp85.r1i1p1.CLMcom-CCLM4-8-17.v1.mon.tas.v20150320'

## Load Catalog for C3S-CMIP5

In [11]:
df_cmip5 = cat['c3s-cmip5'].read()
df_cmip5

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1-m.amip.mon.at...,output1/BCC/bcc-csm1-1-m/amip/mon/atmos/Amon/r...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1-m,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,http://data.mips.copernicus-climate.eu/thredds...,cmip5-monthly-single-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1-m.amip.mon.at...,output1/BCC/bcc-csm1-1-m/amip/mon/atmos/Amon/r...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1-m,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,http://data.mips.copernicus-climate.eu/thredds...,cmip5-monthly-single-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1-m.amip.mon.at...,output1/BCC/bcc-csm1-1-m/amip/mon/atmos/Amon/r...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1-m,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,http://data.mips.copernicus-climate.eu/thredds...,cmip5-monthly-single-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1-m.historical....,output1/BCC/bcc-csm1-1-m/historical/mon/atmos/...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1-m,historical,mon,atmos,...,v20181201,1850-01-01T12:00:00,2012-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,http://data.mips.copernicus-climate.eu/thredds...,cmip5-monthly-single-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1-m.historical....,output1/BCC/bcc-csm1-1-m/historical/mon/atmos/...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1-m,historical,mon,atmos,...,v20181201,1850-01-01T12:00:00,2012-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,http://data.mips.copernicus-climate.eu/thredds...,cmip5-monthly-single-level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136187,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2004-01-01T12:00:00,2004-12-31T12:00:00,NaN,NaN,m.s^-1,Meridional component of Wind (va),Magnitude of the meridional (northward) compon...,http://data.mips.copernicus-climate.eu/thredds...,cmip5-daily-pressure-level
136188,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2005-01-01T12:00:00,2005-12-31T12:00:00,NaN,NaN,m.s^-1,Meridional component of Wind (va),Magnitude of the meridional (northward) compon...,http://data.mips.copernicus-climate.eu/thredds...,cmip5-daily-pressure-level
136189,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2006-01-01T12:00:00,2006-12-31T12:00:00,NaN,NaN,m.s^-1,Meridional component of Wind (va),Magnitude of the meridional (northward) compon...,http://data.mips.copernicus-climate.eu/thredds...,cmip5-daily-pressure-level
136190,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2007-01-01T12:00:00,2007-12-31T12:00:00,NaN,NaN,m.s^-1,Meridional component of Wind (va),Magnitude of the meridional (northward) compon...,http://data.mips.copernicus-climate.eu/thredds...,cmip5-daily-pressure-level


In [12]:
df = df_cmip5.loc[
    (df_cmip5.variable=="tas") 
    & (df_cmip5.experiment=="rcp85")
    & (df_cmip5.time_frequency=="mon")
    & (df_cmip5.model=="bcc-csm1-1-m")
    & (df_cmip5.ensemble=="r1i1p1")
]
df.head()

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
9,c3s-cmip5.output1.BCC.bcc-csm1-1-m.rcp85.mon.a...,output1/BCC/bcc-csm1-1-m/rcp85/mon/atmos/Amon/...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1-m,rcp85,mon,atmos,...,v20181201,2006-01-01T12:00:00,2099-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,http://data.mips.copernicus-climate.eu/thredds...,cmip5-monthly-single-level
10,c3s-cmip5.output1.BCC.bcc-csm1-1-m.rcp85.mon.a...,output1/BCC/bcc-csm1-1-m/rcp85/mon/atmos/Amon/...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1-m,rcp85,mon,atmos,...,v20181201,2100-01-01T12:00:00,2100-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,http://data.mips.copernicus-climate.eu/thredds...,cmip5-monthly-single-level


In [13]:
collection = df.ds_id.unique()[0]
collection

'c3s-cmip5.output1.BCC.bcc-csm1-1-m.rcp85.mon.atmos.Amon.r1i1p1.tas.v20181201'